In [1]:
import numpy as np
from collections import Counter

class Node:
    """
    Represents a node in the decision tree.
    Can be either a decision node (internal) or a leaf node.
    """
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature        # Index of feature to split on
        self.threshold = threshold    # Threshold value for split
        self.left = left             # Left child node
        self.right = right           # Right child node
        self.value = value           # Value if leaf node (prediction)

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    """
    Decision Tree for Classification and Regression

    Parameters:
    -----------
    min_samples_split : int, default=2
        Minimum number of samples required to split a node
    max_depth : int, default=100
        Maximum depth of the tree
    n_features : int, default=None
        Number of features to consider when looking for best split
        If None, then n_features = all features
    task : str, default='classification'
        Type of task: 'classification' or 'regression'
    """

    def __init__(self, min_samples_split=2, max_depth=100, n_features=None, task='classification'):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.task = task
        self.root = None

    def fit(self, X, y):
        """
        Build decision tree from training data

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Target values
        """
        # Determine number of features to use
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)

        # Grow the tree
        # 1- TODO
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        """
        Recursively grow the decision tree
        """
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # Check stopping criteria
        # 2- TODO
        if (depth >= self.max_depth or
            n_labels == 1 or
            n_samples < self.min_samples_split):
            leaf_value = self._leaf_value(y)
            return Node(value=leaf_value)

        # Find best split
        # feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        feat_idxs = np.arange(n_feats)

        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Check if valid split was found
        if best_feature is None:
            return Node(value=self._leaf_value(y))

        # Create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        """
        Find the best feature and threshold to split on
        """
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            #3- TODO
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # Calculate information gain
                gain = self._information_gain(y, X_column, thr)


                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        """
        Calculate information gain from a split
        """
        # Parent impurity
        #4- TODO
        parent_impurity = self._impurity(y)

        # Create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # Calculate weighted avg impurity of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._impurity(y[left_idxs]), self._impurity(y[right_idxs])
        #5- TODO
        child_impurity = self._impurity(y)

        # Calculate information gain
        #6- TODO
        weighted_imp = n_l / n * e_l + n_r / n * e_r
        information_gain = parent_impurity - weighted_imp
        return information_gain

    def _split(self, X_column, split_thresh):
        """
        Split data based on feature value and threshold
        """
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _impurity(self, y):
        """
        Calculate impurity measure based on task type
        """
        if self.task == 'classification':
            # Gini impurity for classification
            hist = np.bincount(y.astype(int))
            #7- TODO
            ps = (hist / len(y))**2
            #8- TODO
            return 1 - np.sum(ps)
        else:
            # Variance for regression
            return np.var(y)

    def _leaf_value(self, y):
        """
        Calculate the value for a leaf node
        """
        if self.task == 'classification':
            # Most common class
            counter = Counter(y)
            value = counter.most_common(1)[0][0]
            return value
        else:
            # Mean value for regression
            return np.mean(y)

    def predict(self, X):
        """
        Predict class or value for X

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Samples to predict

        Returns:
        --------
        predictions : array, shape (n_samples,)
            Predicted values
        """
        #9- TODO
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        """
        Traverse tree to find prediction for a single sample
        """
        #10- TODO
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


# Convenience classes for specific tasks
class DecisionTreeClassifier(DecisionTree):
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        super().__init__(min_samples_split, max_depth, n_features, task='classification')


class DecisionTreeRegressor(DecisionTree):
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        super().__init__(min_samples_split, max_depth, n_features, task='regression')

In [2]:
"""
Simple Manual Example: Understanding Decision Tree Logic
This demonstrates how a decision tree makes decisions step by step
"""

import numpy as np

print("="*60)
print("SIMPLE EXAMPLE: SHOULD WE PLAY TENNIS?")
print("="*60)


# High humidity: 85-96%
# Normal humidity: 65-80%
X_train = np.array([
    [0, 85, 85, 0],  # not Rainy, Hot, High humidity, Weak wind -> No
    [0, 80, 90, 1],  # not Rainy, Hot, High humidity, Strong wind -> No
    [0, 83, 78, 0],  # not Rainy, Hot, High humidity, Weak wind -> Yes
    [1, 70, 96, 0],  # Rainy, Mild, High humidity, Weak wind -> Yes
    [1, 68, 80, 0],  # Rainy, Cool, Normal humidity, Weak wind -> Yes
    [1, 65, 70, 1],  # Rainy, Cool, Normal humidity, Strong wind -> No
    [0, 64, 65, 1],  # not Rainy, Cool, Normal humidity, Strong wind -> Yes
    [0, 72, 95, 0],  # not Rainy, Mild, High humidity, Weak wind -> No
    [0, 69, 70, 0],  # not Rainy, Cool, Normal humidity, Weak wind -> Yes
    [1, 75, 80, 0],  # Rainy, Mild, Normal humidity, Weak wind -> Yes
    [0, 75, 70, 1],  # not Rainy, Mild, Normal humidity, Strong wind -> Yes
    [0, 72, 90, 1],  # not Rainy, Mild, High humidity, Strong wind -> Yes
    [0, 81, 75, 0],  # not Rainy, Hot, Normal humidity, Weak wind -> Yes
    [1, 71, 80, 1],  # Rainy, Mild, Normal humidity, Strong wind -> No
])

y_train = np.array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])

feature_names = ['Outlook', 'Temperature', 'Humidity', 'Wind']

# min_samples_split = 2:
#   - A node with 2+ samples can be split
#   - Very permissive, tree grows large
#   - Risk of overfitting

# min_samples_split = 10:
#   - A node needs at least 10 samples to split
#   - More restrictive, tree stays smaller
#   - Prevents overfitting

# Train the tree
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=2)
clf.fit(X_train, y_train)

print("\nTraining completed!")
print(f"Total training samples: {len(y_train)}")
print(f"Play Tennis: {sum(y_train)} times")
print(f"Don't Play: {len(y_train) - sum(y_train)} times")


# Test predictions
print("\n" + "="*60)
print("MAKING PREDICTIONS")
print("="*60)

test_cases = [
    ([0, 70, 80, 0], "not Rainy, Cool, High Humidity, Weak Wind"),
    ([0, 75, 75, 0], "not Rainy, Mild, Normal Humidity, Weak Wind"),
    ([1, 65, 90, 1], "Rainy, Cool, High Humidity, Strong Wind"),
]

for features, description in test_cases:
    prediction = clf.predict(np.array([features]))[0]
    result = "Yes, Play Tennis! ✓" if prediction == 1 else "No, Don't Play ✗"
    print(f"\nWeather: {description}")
    print(f"Features: {features}")
    print(f"Decision: {result}")





SIMPLE EXAMPLE: SHOULD WE PLAY TENNIS?

Training completed!
Total training samples: 14
Play Tennis: 9 times
Don't Play: 5 times

MAKING PREDICTIONS

Weather: not Rainy, Cool, High Humidity, Weak Wind
Features: [0, 70, 80, 0]
Decision: Yes, Play Tennis! ✓

Weather: not Rainy, Mild, Normal Humidity, Weak Wind
Features: [0, 75, 75, 0]
Decision: Yes, Play Tennis! ✓

Weather: Rainy, Cool, High Humidity, Strong Wind
Features: [1, 65, 90, 1]
Decision: Yes, Play Tennis! ✓
